Load API Keys

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

Load Contents from HLB Fixed Deposite Pages

In [2]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader([
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit.html?icp=hlb-en-all-footer-txt-fd",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/fixed-deposit-account.html", 
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/e-fixed-deposit.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/flexi-fd.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/senior-savers-flexi-fd.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/junior-fixed-deposit.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/foreign-fixed-deposit-account.html",
  "https://www.hlb.com.my/en/personal-banking/help-support/fees-and-charges/deposits.html",
])
data = loader.load()
data

[Document(page_content="\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nFixed Deposit - HLB\n\n\n\n\n\n\n\n\n\n\nFixed Deposit - HLB\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPERSONAL\n\n\nBUSINESS\n\n\n\n                            Priority\n\n\nPriority\nIslamic\nGlobal Markets\nBlue Header\nInterim Homepage\nPriority Banking Form\ntest\ntest\nLoan Cal\nLog Viewer\nasdsadsad\nstand alone page\nMaster Form Page\nMaster Form Page\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nsingapore\n\n\n\nhong kong\n\n\n\nvietnam\n\n\n\ncambodia\n\n\n\n\n\n                            ENG\n\n\nLAN A\nLAN B\nLAN C\n\n\n\n\n                            Online Banking\n                        \n\nPersonal\nBusiness\n\n\n\n\n\n\n\n\n\n\n\n\n\n                                About Us\n                            \n\nOur Story\nNews & Updates\nInvestor Relations\nCSR\nAwards\nManagement & BODs\nCareer\n\n\n\n\n                                Discover Our Products\n      

Split the Loaded Data into Chunks

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
texts = text_splitter.split_documents(data)
texts

[Document(page_content='Fixed Deposit - HLB\n\n\n\n\n\n\n\n\n\n\nFixed Deposit - HLB\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPERSONAL\n\n\nBUSINESS\n\n\n\n                            Priority\n\n\nPriority\nIslamic\nGlobal Markets\nBlue Header\nInterim Homepage\nPriority Banking Form\ntest\ntest\nLoan Cal\nLog Viewer\nasdsadsad\nstand alone page\nMaster Form Page\nMaster Form Page\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nsingapore\n\n\n\nhong kong\n\n\n\nvietnam\n\n\n\ncambodia\n\n\n\n\n\n                            ENG\n\n\nLAN A\nLAN B\nLAN C\n\n\n\n\n                            Online Banking\n                        \n\nPersonal\nBusiness\n\n\n\n\n\n\n\n\n\n\n\n\n\n                                About Us\n                            \n\nOur Story\nNews & Updates\nInvestor Relations\nCSR\nAwards\nManagement & BODs\nCareer\n\n\n\n\n                                Discover Our Products\n                            \n\nDeposits\nCredit Card\nLoans\nInsurance\nInvestment\nRemittance\

Initialize Pinecone

In [4]:
import os
import pinecone

pinecone.init(
  api_key=os.getenv("PINECONE_API_KEY"),
  environment=os.getenv("PINECONE_ENVIRONMENT")
)

index_name = "main-index"
namespace = "hlb-fixed-deposit-with-cohere"

if index_name not in pinecone.list_indexes():
  pinecone.create_index(
    name=index_name,
    metric="cosine",
    dimension=1536
  )
  
index = pinecone.Index(index_name)
index.delete(delete_all=True, namespace=namespace)

d:\Code\langchain-test\venv\Lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


{}

Create Embeddings and Vectors to Insert to Pinecone Database

In [5]:
from langchain.vectorstores import Pinecone
from langchain.embeddings import CohereEmbeddings, OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
vector = Pinecone.from_documents(texts, embeddings, index_name=index_name, namespace=namespace)

Create Retriever for Vector Search

In [42]:
retriever = vector.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.9, "k": 5})
retriever

VectorStoreRetriever(tags=['Pinecone', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.pinecone.Pinecone object at 0x0000023C90BA3490>, search_type='similarity_score_threshold', search_kwargs={'score_threshold': 0.9, 'k': 5})

Creating Prompt Template

In [43]:
from langchain.prompts import PromptTemplate
from langchain.llms import Replicate

llm = Replicate(model="meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3", model_kwrags ={"temperature": 0})
prompt_template = """
  You are a helpful assistant who should reply to inputs related only to the HongLeong Bank aka HLB.
  You answer to the question: {question}
  By only basing on this context: {context}
  If you don't have enought information from the document, don't answer with your general knowledge, just say you don't have that information.
  Your answer should be detailed but if you have to do calculation, don't show your work or equations, double check your answer, just provide the final value, and you don't need to show reference.
  If the bank's name is not provided in the question, answer with the assumption that the question is about HLB.
  If the bank's name is provided and it's not HLB, don't answer that question.
  You don't need to answer if the input question you to do something creative such as writing a song or making a joke.
"""
prompt = PromptTemplate(template=prompt_template, input_variables=['question', 'context'])

model_kwrags was transferred to model_kwargs.
                    Please confirm that model_kwrags is what you intended.


Creating Memory

In [44]:
from langchain.memory import ConversationSummaryBufferMemory

memory = ConversationSummaryBufferMemory(llm=llm, memory_key="chat_history", output_key="answer", max_token_limit=500, return_messages=True)

Creating Chain

In [45]:
from langchain.chains import ConversationalRetrievalChain
from langchain.callbacks import get_openai_callback

chain = ConversationalRetrievalChain.from_llm(
  llm=llm,
  chain_type="stuff",
  retriever=retriever,
  memory=memory,
  verbose=True,
  combine_docs_chain_kwargs={"prompt": prompt},
  return_source_documents=True
)

def ask_question(question):
  with get_openai_callback() as cb:
    result = chain(question)
    print(f'{cb}\n')
    print(f'\nAnswer: {result["answer"]}')
    for source_documents in result["source_documents"]:
      print(f'\nDocument: {source_documents}')


Running the Chain

In [40]:
ask_question("What is fixed deposit?")



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

system:  The human asked about the different types of fixed deposit accounts provided by Hong Leong Bank, and the AI replied with the three types of accounts they offer, which include eFixed Deposit, Foreign Currency Fixed Deposit (FCFD), and Flexi FD. The human then inquired about the distinction between a traditional fixed deposit and an eFixed Deposit. The AI explained two main differences, which are the interest rate and tenure. eFixed Deposit typically offers higher interest rates as it's designed to be more flexible and liquid, and traditional fixed deposits have
Human: Lets say I want to invest RM 50,000 in Fixed Deposit for 12 months. Please calculate the total amount that I can withdraw at the end of the term.
Assistant:  Sure, I'd be happy to help! Based on 

In [34]:
ask_question("How many types of fixed deposit does HongLeong Bank provide?")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

  You are a helpful assistant who should reply to inputs related only to the HongLeong Bank aka HLB.
  You answer to the question: How many types of fixed deposit does HongLeong Bank provide?
  By only basing on this context: Fixed Deposit | Hong Leong Bank Malaysia










Fixed Deposit
 
null























PERSONAL


BUSINESS



                            Priority


Priority
Islamic
Global Markets
Blue Header
Interim Homepage
Priority Banking Form
test
test
Loan Cal
Log Viewer
asdsadsad
stand alone page
Master Form Page
Master Form Page
















singapore



hong kong



vietnam



cambodia





                            ENG


LAN A
LAN B
LAN C




                            Online Banking
                        

Personal
Business













                                About Us
                            

Our Story
News & Updates
Investor Relation

In [35]:
ask_question("What is the difference between Fixed Deposit and eFixed Deposit?")



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: How many types of fixed deposit does HongLeong Bank provide?
Assistant:  Based on the information provided, Hong Leong Bank provides three types of fixed deposit accounts:

1. eFixed Deposit
2. Foreign Currency Fixed Deposit (FCFD)
3. Flexi FD

These are the three types of fixed deposit accounts offered by Hong Leong Bank.
Follow Up Input: What is the difference between Fixed Deposit and eFixed Deposit?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

  You are a helpful assistant who should reply to inputs related only to the HongLeong Bank aka HLB.
  You answer to the question:  Sure! Here's a standalone question based on the follow-up input:

What are the key di

In [ ]:
ask_question("What are the interest rates for normal Fixed Deposit?")

In [47]:
ask_question("Lets say I want to invest RM 50,000 in Fixed Deposit for 12 months. Please calculate the total amount that I can withdraw at the end of the term.")



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: who is jay z?
Assistant:  Understood! I'm ready to assist with any questions you may have related to Hong Leong Bank (HLB).

Who is Jay Z?

I don't have enough information to provide an answer to this question. Could you please provide more context or clarify which Jay Z you are referring to?
Follow Up Input: Lets say I want to invest RM 50,000 in Fixed Deposit for 12 months. Please calculate the total amount that I can withdraw at the end of the term.
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

  You are a helpful assistant who should reply to inputs related only to the HongLeong Bank aka HLB.
  You answer to the question:  Sure, here's a standalone question 

In [46]:
ask_question("who is jay z?")

d:\Code\langchain-test\venv\Lib\site-packages\langchain_core\vectorstores.py:325: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.9
  warnings.warn(




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

  You are a helpful assistant who should reply to inputs related only to the HongLeong Bank aka HLB.
  You answer to the question: who is jay z?
  By only basing on this context: 
  If you don't have enought information from the document, don't answer with your general knowledge, just say you don't have that information.
  Your answer should be detailed but if you have to do calculation, don't show your work or equations, double check your answer, just provide the final value, and you don't need to show reference.
  If the bank's name is not provided in the question, answer with the assumption that the question is about HLB.
  If the bank's name is provided and it's not HLB, don't answer that question.
  You don't need to answer if the input question you to do something creative such as writing a song or making a joke.


> Finished chain.

> Finished chain.
Tokens Used: 0
	Prompt 